---
title: "動き出す PDMP"
subtitle: "`PDMPFlux.jl` のアニメーションで見る MCMC の最近の発展"
image: Emergence/PDMPFlux.png
author: "司馬博文"
date: "12/24/2025"
categories: [PDMP, MCMC, Process]
bibliography: 
    - ../../../assets/2023.bib
    - ../../../assets/2024.bib
    - ../../../assets/2025.bib
csl: ../../../assets/apalike.csl
---


## はじめに

PDMP (Piecewise Deterministic Markov Process) とは近年さかんに研究されている全く新しい MCMC の総称である．

収束が速い，大規模データに特に強い，密度が非正則だったりアトムを持っていても良いなど，理論的な期待は大きいが，その動いている様子を見たことのある者は，まだまだ一部の研究者に限られる．

HMC が Stan や PyMC に高度な実装があり，日々の統計と機械学習の実践を支えているのとは対照的である．

ここでは筆者が開発している Julia パッケージ [`PDMPFlux.jl`](https://github.com/162348/PDMPFlux.jl) を紹介する．

その最大の特徴は柔軟なアニメーション機能を提供する関数 `anim_traj` である．これにより種々の PDMP サンプラーの動きをアニメーションで見ることができる．本稿ではこの関数をフルに活用して，PDMP を「図解」してみたい．

PDMP はシミュレーション方法も未だ定まっておらず，実用上も理論上もまだまだ発展途上である．実は本パッケージは [@Andral-Kamatani2024] の実装という点が特徴であるが，PDMP のアルゴリズムの側面には深入りしすぎず，本稿の終わりには PDMP の理論的な側面を少し紹介したい．

突然だが，クイズである．次クイズに答えるつもりで本稿を読んでみてほしい．

以下に，本パッケージが提供する PDMP サンプラー６つのうち３つを示す．このうち，**最も高次元空間に強い手法はどれだろうか？**

:::: {layout-ncol="3"}

![[@Bouchard-Cote+2018]](../../2024/Slides/PDMPs/BPS_SlantedGauss2D.gif)

![[@Bierkens+2019]](../../2024/Slides/PDMPs/ZigZag_SlantedGauss2D.gif)

![[@Michel+2020]](../../2024/Slides/PDMPs/ForwardECMC_StandardGauss2D.gif)

::::


::: {.hidden}

::: {.content-visible when-format="html"}

A Blog Entry on Bayesian Computation by an Applied Mathematician

$$

\renewcommand{\P}{\operatorname{P}}\newcommand{\E}{\operatorname{E}}
\newcommand{\R}{\mathbb{R}}\newcommand{\bR}{\mathbb{R}}\newcommand{\F}{\mathcal{F}}
\newcommand{\abs}[1]{\lvert#1\rvert}\newcommand{\Abs}[1]{\left|#1\right|}\newcommand{\ABs}[1]{\biggl|#1\biggr|}\newcommand{\norm}[1]{\|#1\|}\newcommand{\Norm}[1]{\left\|#1\right\|}\newcommand{\NOrm}[1]{\biggl\|#1\biggr\|}\newcommand{\Brace}[1]{\left\{#1\right\}}\newcommand{\BRace}[1]{\biggl\{#1\biggr\}}\newcommand{\paren}[1]{\left(#1\right)}\newcommand{\Paren}[1]{\biggr(#1\biggl)}\newcommand{\brac}[1]{\langle#1\rangle}\newcommand{\Brac}[1]{\left\langle#1\right\rangle}\newcommand{\BRac}[1]{\biggl\langle#1\biggr\rangle}\newcommand{\bra}[1]{\left\langle#1\right|}\newcommand{\ket}[1]{\left|#1\right\rangle}\newcommand{\Square}[1]{\left[#1\right]}\newcommand{\SQuare}[1]{\biggl[#1\biggr]}\newcommand{\rN}{\operatorname{N}}\newcommand{\ov}[1]{\overline{#1}}\newcommand{\un}[1]{\underline{#1}}\newcommand{\wt}[1]{\widetilde{#1}}\newcommand{\wh}[1]{\widehat{#1}}\newcommand{\pp}[2]{\frac{\partial #1}{\partial #2}}\newcommand{\ppp}[3]{\frac{\partial #1}{\partial #2\partial #3}}\newcommand{\dd}[2]{\frac{d #1}{d #2}}\newcommand{\floor}[1]{\lfloor#1\rfloor}\newcommand{\Floor}[1]{\left\lfloor#1\right\rfloor}\newcommand{\ceil}[1]{\lceil#1\rceil}\newcommand{\ocinterval}[1]{(#1]}\newcommand{\cointerval}[1]{[#1)}\newcommand{\COinterval}[1]{\left[#1\right)}\newcommand{\iso}{\overset{\sim}{\to}}



\newcommand{\y}{\b{y}}\newcommand{\mi}{\,|\,}\newcommand{\Mark}{\mathrm{Mark}}
\newcommand{\argmax}{\operatorname*{argmax}}\newcommand{\argmin}{\operatorname*{argmin}}

\newcommand{\pr}{\mathrm{pr}}\newcommand{\Conv}{\operatorname{Conv}}\newcommand{\cU}{\mathcal{U}}
\newcommand{\Map}{\mathrm{Map}}\newcommand{\dom}{\mathrm{Dom}\;}\newcommand{\cod}{\mathrm{Cod}\;}\newcommand{\supp}{\mathrm{supp}\;}
\newcommand{\grad}{\operatorname{grad}}\newcommand{\rot}{\operatorname{rot}}\renewcommand{\div}{\operatorname{div}}\newcommand{\tr}{\operatorname{tr}}\newcommand{\Tr}{\operatorname{Tr}}\newcommand{\KL}{\operatorname{KL}}\newcommand{\JS}{\operatorname{JS}}\newcommand{\ESS}{\operatorname{ESS}}\newcommand{\MSE}{\operatorname{MSE}}\newcommand{\erf}{\operatorname{erf}}\newcommand{\arctanh}{\operatorname{arctanh}}\newcommand{\pl}{\operatorname{pl}}\newcommand{\minimize}{\operatorname{minimize}}\newcommand{\subjectto}{\operatorname{subject to}}\newcommand{\sinc}{\operatorname{sinc}}\newcommand{\Ent}{\operatorname{Ent}}\newcommand{\Polya}{\operatorname{Polya}}\newcommand{\Exp}{\operatorname{Exp}}\newcommand{\codim}{\operatorname{codim}}\newcommand{\sgn}{\operatorname{sgn}}\newcommand{\rank}{\operatorname{rank}}

\newcommand{\vctr}[2]{\begin{pmatrix}#1\\#2\end{pmatrix}}\newcommand{\vctrr}[3]{\begin{pmatrix}#1\\#2\\#3\end{pmatrix}}\newcommand{\mtrx}[4]{\begin{pmatrix}#1&#2\\#3&#4\end{pmatrix}}\newcommand{\smtrx}[4]{\paren{\begin{smallmatrix}#1&#2\\#3&#4\end{smallmatrix}}}\newcommand{\Ker}{\mathrm{Ker}\;}\newcommand{\Coker}{\mathrm{Coker}\;}\newcommand{\Coim}{\mathrm{Coim}\;}\newcommand{\lcm}{\mathrm{lcm}}\newcommand{\GL}{\mathrm{GL}}\newcommand{\SL}{\mathrm{SL}}\newcommand{\alt}{\mathrm{alt}}

\renewcommand{\Re}{\mathrm{Re}\;}\renewcommand{\Im}{\mathrm{Im}\,}\newcommand{\Gal}{\mathrm{Gal}}\newcommand{\PGL}{\mathrm{PGL}}\newcommand{\PSL}{\mathrm{PSL}}\newcommand{\Log}{\mathrm{Log}\,}\newcommand{\Res}{\mathrm{Res}\,}\newcommand{\on}{\mathrm{on}\;}\newcommand{\hatC}{\widehat{\C}}\newcommand{\hatR}{\hat{\R}}\newcommand{\PV}{\mathrm{P.V.}}\newcommand{\diam}{\mathrm{diam}}\newcommand{\Area}{\mathrm{Area}}\newcommand{\Lap}{\Laplace}\newcommand{\f}{\mathbf{f}}\newcommand{\cR}{\mathcal{R}}\newcommand{\const}{\mathrm{const.}}\newcommand{\Om}{\Omega}\newcommand{\Cinf}{C^\infty}\newcommand{\ep}{\epsilon}\newcommand{\dist}{\mathrm{dist}}\newcommand{\opart}{\o{\partial}}\newcommand{\Length}{\mathrm{Length}}

\newcommand{\cA}{\mathcal{A}}\newcommand{\cO}{\mathcal{O}}\newcommand{\cW}{\mathcal{W}}\renewcommand{\O}{\mathcal{O}}\renewcommand{\S}{\mathcal{S}}\newcommand{\U}{\mathcal{U}}\newcommand{\V}{\mathrm{V}}\newcommand{\N}{\mathbb{N}}\newcommand{\bN}{\mathbb{N}}\newcommand{\C}{\mathrm{C}}\newcommand{\bC}{\mathbb{C}}\newcommand{\Z}{\mathcal{Z}}\newcommand{\Q}{\mathbb{Q}}\newcommand{\bQ}{\mathbb{Q}}\newcommand{\TV}{\mathrm{TV}}\newcommand{\ORD}{\mathrm{ORD}}\newcommand{\Card}{\mathrm{Card}\,}\newcommand{\Top}{\mathrm{Top}}\newcommand{\Disc}{\mathrm{Disc}}\newcommand{\Codisc}{\mathrm{Codisc}}\newcommand{\CoDisc}{\mathrm{CoDisc}}\newcommand{\Ult}{\mathrm{Ult}}\newcommand{\ord}{\mathrm{ord}}\newcommand{\bS}{\mathbb{S}}\newcommand{\PConn}{\mathrm{PConn}}\newcommand{\mult}{\mathrm{mult}}\newcommand{\inv}{\mathrm{inv}}

\newcommand{\Der}{\mathrm{Der}}\newcommand{\osub}{\overset{\mathrm{open}}{\subset}}\newcommand{\osup}{\overset{\mathrm{open}}{\supset}}\newcommand{\al}{\alpha}\newcommand{\K}{\mathbb{K}}\newcommand{\Sp}{\mathrm{Sp}}\newcommand{\g}{\mathfrak{g}}\newcommand{\h}{\mathfrak{h}}\newcommand{\Imm}{\mathrm{Imm}}\newcommand{\Imb}{\mathrm{Imb}}\newcommand{\Gr}{\mathrm{Gr}}

\newcommand{\Ad}{\mathrm{Ad}}\newcommand{\finsupp}{\mathrm{fin\;supp}}\newcommand{\SO}{\mathrm{SO}}\newcommand{\SU}{\mathrm{SU}}\newcommand{\acts}{\curvearrowright}\newcommand{\mono}{\hookrightarrow}\newcommand{\epi}{\twoheadrightarrow}\newcommand{\Stab}{\mathrm{Stab}}\newcommand{\nor}{\mathrm{nor}}\newcommand{\T}{\mathbb{T}}\newcommand{\Aff}{\mathrm{Aff}}\newcommand{\rsup}{\triangleright}\newcommand{\subgrp}{\overset{\mathrm{subgrp}}{\subset}}\newcommand{\Ext}{\mathrm{Ext}}\newcommand{\sbs}{\subset}\newcommand{\sps}{\supset}\newcommand{\In}{\mathrm{in}\;}\newcommand{\Tor}{\mathrm{Tor}}\newcommand{\p}{\b{p}}\newcommand{\q}{\mathfrak{q}}\newcommand{\m}{\mathfrak{m}}\newcommand{\cS}{\mathcal{S}}\newcommand{\Frac}{\mathrm{Frac}\,}\newcommand{\Spec}{\mathrm{Spec}\,}\newcommand{\bA}{\mathbb{A}}\newcommand{\Sym}{\mathrm{Sym}}\newcommand{\Ann}{\mathrm{Ann}}\newcommand{\Her}{\mathrm{Her}}\newcommand{\Bil}{\mathrm{Bil}}\newcommand{\Ses}{\mathrm{Ses}}\newcommand{\FVS}{\mathrm{FVS}}

\newcommand{\Ho}{\mathrm{Ho}}\newcommand{\CW}{\mathrm{CW}}\newcommand{\lc}{\mathrm{lc}}\newcommand{\cg}{\mathrm{cg}}\newcommand{\Fib}{\mathrm{Fib}}\newcommand{\Cyl}{\mathrm{Cyl}}\newcommand{\Ch}{\mathrm{Ch}}
\newcommand{\rP}{\mathrm{P}}\newcommand{\rE}{\mathrm{E}}\newcommand{\e}{\b{e}}\renewcommand{\k}{\b{k}}\newcommand{\Christ}[2]{\begin{Bmatrix}#1\\#2\end{Bmatrix}}\renewcommand{\Vec}[1]{\overrightarrow{\mathrm{#1}}}\newcommand{\hen}[1]{\mathrm{#1}}\renewcommand{\b}[1]{\boldsymbol{#1}}

\newcommand{\Inc}{\mathrm{Inc}}\newcommand{\aInc}{\mathrm{aInc}}\newcommand{\HS}{\mathrm{HS}}\newcommand{\loc}{\mathrm{loc}}\newcommand{\Lh}{\mathrm{L.h.}}\newcommand{\Epi}{\mathrm{Epi}}\newcommand{\slim}{\mathrm{slim}}\newcommand{\Ban}{\mathrm{Ban}}\newcommand{\Hilb}{\mathrm{Hilb}}\newcommand{\Ex}{\mathrm{Ex}}\newcommand{\Co}{\mathrm{Co}}\newcommand{\sa}{\mathrm{sa}}\newcommand{\nnorm}[1]{{\left\vert\kern-0.25ex\left\vert\kern-0.25ex\left\vert #1 \right\vert\kern-0.25ex\right\vert\kern-0.25ex\right\vert}}\newcommand{\dvol}{\mathrm{dvol}}\newcommand{\Sconv}{\mathrm{Sconv}}\newcommand{\I}{\mathcal{I}}\newcommand{\nonunital}{\mathrm{nu}}\newcommand{\cpt}{\mathrm{cpt}}\newcommand{\lcpt}{\mathrm{lcpt}}\newcommand{\com}{\mathrm{com}}\newcommand{\Haus}{\mathrm{Haus}}\newcommand{\proper}{\mathrm{proper}}\newcommand{\infinity}{\mathrm{inf}}\newcommand{\TVS}{\mathrm{TVS}}\newcommand{\ess}{\mathrm{ess}}\newcommand{\ext}{\mathrm{ext}}\newcommand{\Index}{\mathrm{Index}\;}\newcommand{\SSR}{\mathrm{SSR}}\newcommand{\vs}{\mathrm{vs.}}\newcommand{\fM}{\mathfrak{M}}\newcommand{\EDM}{\mathrm{EDM}}\newcommand{\Tw}{\mathrm{Tw}}\newcommand{\fC}{\mathfrak{C}}\newcommand{\bn}{\boldsymbol{n}}\newcommand{\br}{\boldsymbol{r}}\newcommand{\Lam}{\Lambda}\newcommand{\lam}{\lambda}\newcommand{\one}{\mathbf{1}}\newcommand{\dae}{\text{-a.e.}}\newcommand{\das}{\text{-a.s.}}\newcommand{\td}{\text{-}}\newcommand{\RM}{\mathrm{RM}}\newcommand{\BV}{\mathrm{BV}}\newcommand{\normal}{\mathrm{normal}}\newcommand{\lub}{\mathrm{lub}\;}\newcommand{\Graph}{\mathrm{Graph}}\newcommand{\Ascent}{\mathrm{Ascent}}\newcommand{\Descent}{\mathrm{Descent}}\newcommand{\BIL}{\mathrm{BIL}}\newcommand{\fL}{\mathfrak{L}}\newcommand{\De}{\Delta}

\newcommand{\calA}{\mathcal{A}}\newcommand{\calB}{\mathcal{B}}\newcommand{\D}{\mathcal{D}}\newcommand{\Y}{\mathcal{Y}}\newcommand{\calC}{\mathcal{C}}\renewcommand{\ae}{\mathrm{a.e.}\;}\newcommand{\cZ}{\mathcal{Z}}\newcommand{\fF}{\mathfrak{F}}\newcommand{\fI}{\mathfrak{I}}\newcommand{\rV}{\mathrm{V}}\newcommand{\cE}{\mathcal{E}}\newcommand{\sMap}{\sigma\textrm{-}\mathrm{Map}}\newcommand{\cC}{\mathcal{C}}\newcommand{\comp}{\complement}\newcommand{\J}{\mathcal{J}}\newcommand{\sumN}[1]{\sum_{#1\in\N}}\newcommand{\cupN}[1]{\cup_{#1\in\N}}\newcommand{\capN}[1]{\cap_{#1\in\N}}\newcommand{\Sum}[1]{\sum_{#1=1}^\infty}\newcommand{\sumn}{\sum_{n=1}^\infty}\newcommand{\summ}{\sum_{m=1}^\infty}\newcommand{\sumk}{\sum_{k=1}^\infty}\newcommand{\sumi}{\sum_{i=1}^\infty}\newcommand{\sumj}{\sum_{j=1}^\infty}\newcommand{\cupn}{\cup_{n=1}^\infty}\newcommand{\capn}{\cap_{n=1}^\infty}\newcommand{\cupk}{\cup_{k=1}^\infty}\newcommand{\cupi}{\cup_{i=1}^\infty}\newcommand{\cupj}{\cup_{j=1}^\infty}\newcommand{\limn}{\lim_{n\to\infty}}\renewcommand{\L}{\mathcal{L}}\newcommand{\cL}{\mathcal{L}}\newcommand{\Cl}{\mathrm{Cl}}\newcommand{\cN}{\mathcal{N}}\newcommand{\Ae}{\textrm{-a.e.}\;}\renewcommand{\csub}{\overset{\textrm{closed}}{\subset}}\renewcommand{\csup}{\overset{\textrm{closed}}{\supset}}\newcommand{\wB}{\wt{B}}\newcommand{\cG}{\mathcal{G}}\newcommand{\Lip}{\mathrm{Lip}}\newcommand{\AC}{\mathrm{AC}}\newcommand{\Mol}{\mathrm{Mol}}

\newcommand{\Pe}{\mathrm{Pe}}\newcommand{\wR}{\wh{\mathbb{\R}}}\newcommand*{\Laplace}{\mathop{}\!\mathbin\bigtriangleup}\newcommand*{\DAlambert}{\mathop{}\!\mathbin\Box}\newcommand{\bT}{\mathbb{T}}\newcommand{\dx}{\dslash x}\newcommand{\dt}{\dslash t}\newcommand{\ds}{\dslash s}

\newcommand{\round}{\mathrm{round}}\newcommand{\cond}{\mathrm{cond}}\newcommand{\diag}{\mathrm{diag}}
\newcommand{\Adj}{\mathrm{Adj}}\newcommand{\Pf}{\mathrm{Pf}}\newcommand{\Sg}{\mathrm{Sg}}


\newcommand{\aseq}{\overset{\text{a.s.}}{=}}\newcommand{\deq}{\overset{\text{d}}{=}}\newcommand{\cV}{\mathcal{V}}\newcommand{\FM}{\mathrm{FM}}\newcommand{\KR}{\mathrm{KR}}\newcommand{\rba}{\mathrm{rba}}\newcommand{\rca}{\mathrm{rca}}\newcommand{\Prob}{\mathrm{Prob}}\newcommand{\X}{\mathcal{X}}\newcommand{\Meas}{\mathrm{Meas}}\newcommand{\as}{\;\text{a.s.}}\newcommand{\io}{\;\mathrm{i.o.}}\newcommand{\fe}{\;\text{f.e.}}\newcommand{\bF}{\mathbb{F}}\newcommand{\W}{\mathcal{W}}\newcommand{\Pois}{\mathrm{Pois}}\newcommand{\iid}{\text{i.i.d.}}\newcommand{\wconv}{\rightsquigarrow}\newcommand{\Var}{\mathrm{Var}}\newcommand{\xrightarrown}{\xrightarrow{n\to\infty}}\newcommand{\au}{\mathrm{au}}\newcommand{\cT}{\mathcal{T}}\newcommand{\wto}{\overset{\text{w}}{\to}}\newcommand{\dto}{\overset{\text{d}}{\to}}\newcommand{\sto}{\overset{\text{s}}{\to}}\newcommand{\pto}{\overset{\text{p}}{\to}}\newcommand{\mto}{\overset{\text{m}}{\to}}\newcommand{\vto}{\overset{v}{\to}}\newcommand{\Cont}{\mathrm{Cont}}\newcommand{\stably}{\mathrm{stably}}\newcommand{\Np}{\mathbb{N}^+}\newcommand{\oM}{\overline{\mathcal{M}}}\newcommand{\fP}{\mathfrak{P}}\newcommand{\sign}{\mathrm{sign}}
\newcommand{\Borel}{\mathrm{Borel}}\newcommand{\Mid}{\,|\,}\newcommand{\middleMid}{\;\middle|\;}\newcommand{\CP}{\mathrm{CP}}\newcommand{\bD}{\mathbb{D}}\newcommand{\bL}{\mathbb{L}}\newcommand{\fW}{\mathfrak{W}}\newcommand{\DL}{\mathcal{D}\mathcal{L}}\renewcommand{\r}[1]{\mathrm{#1}}\newcommand{\rC}{\mathrm{C}}\newcommand{\qqquad}{\qquad\quad}

\newcommand{\bit}{\mathrm{bit}}

\newcommand{\err}{\mathrm{err}}

\newcommand{\varparallel}{\mathbin{\!/\mkern-5mu/\!}}\newcommand{\Ri}{\mathrm{Ri}}\newcommand{\Cone}{\mathrm{Cone}}\newcommand{\Int}{\mathrm{Int}}

\newcommand{\pre}{\mathrm{pre}}\newcommand{\om}{\omega}


\newcommand{\del}{\partial}
\newcommand{\LHS}{\mathrm{LHS}}\newcommand{\RHS}{\mathrm{RHS}}\newcommand{\bnu}{\boldsymbol{\nu}}\newcommand{\interior}{\mathrm{in}\;}\newcommand{\SH}{\mathrm{SH}}\renewcommand{\v}{\boldsymbol{\nu}}\newcommand{\n}{\mathbf{n}}\newcommand{\ssub}{\Subset}\newcommand{\curl}{\mathrm{curl}}

\newcommand{\Ei}{\mathrm{Ei}}\newcommand{\sn}{\mathrm{sn}}\newcommand{\wgamma}{\widetilde{\gamma}}

\newcommand{\Ens}{\mathrm{Ens}}

\newcommand{\cl}{\mathrm{cl}}\newcommand{\x}{\boldsymbol{x}}

\newcommand{\Do}{\mathrm{Do}}\newcommand{\IV}{\mathrm{IV}}

\newcommand{\AIC}{\mathrm{AIC}}\newcommand{\mrl}{\mathrm{mrl}}\newcommand{\dotx}{\dot{x}}\newcommand{\UMV}{\mathrm{UMV}}\newcommand{\BLU}{\mathrm{BLU}}

\newcommand{\comb}[2]{\begin{pmatrix}#1\\#2\end{pmatrix}}\newcommand{\bP}{\mathbb{P}}\newcommand{\compsub}{\overset{\textrm{cpt}}{\subset}}\newcommand{\lip}{\textrm{lip}}\newcommand{\BL}{\mathrm{BL}}\newcommand{\G}{\mathbb{G}}\newcommand{\NB}{\mathrm{NB}}\newcommand{\oR}{\ov{\R}}\newcommand{\liminfn}{\liminf_{n\to\infty}}\newcommand{\limsupn}{\limsup_{n\to\infty}}\newcommand{\esssup}{\mathrm{ess.sup}}\newcommand{\asto}{\xrightarrow{\as}}\newcommand{\Cov}{\mathrm{Cov}}\newcommand{\cQ}{\mathcal{Q}}\newcommand{\VC}{\mathrm{VC}}\newcommand{\mb}{\mathrm{mb}}\newcommand{\Avar}{\mathrm{Avar}}\newcommand{\bB}{\mathbb{B}}\newcommand{\bW}{\mathbb{W}}\newcommand{\sd}{\mathrm{sd}}\newcommand{\w}[1]{\widehat{#1}}\newcommand{\bZ}{\mathbb{Z}}\newcommand{\Bernoulli}{\mathrm{Ber}}\newcommand{\Ber}{\mathrm{Ber}}\newcommand{\Mult}{\mathrm{Mult}}\newcommand{\BPois}{\mathrm{BPois}}\newcommand{\fraks}{\mathfrak{s}}\newcommand{\frakk}{\mathfrak{k}}\newcommand{\IF}{\mathrm{IF}}\newcommand{\bX}{\boldsymbol{X}}\newcommand{\bx}{\boldsymbol{x}}\newcommand{\indep}{\perp\!\!\!\perp}\newcommand{\IG}{\mathrm{IG}}\newcommand{\Levy}{\mathrm{Levy}}\newcommand{\MP}{\mathrm{MP}}\newcommand{\Hermite}{\mathrm{Hermite}}\newcommand{\Skellam}{\mathrm{Skellam}}\newcommand{\Dirichlet}{\mathrm{Dirichlet}}\renewcommand{\Beta}{\operatorname{Beta}}\newcommand{\bE}{\mathbb{E}}\newcommand{\bG}{\mathbb{G}}\newcommand{\MISE}{\mathrm{MISE}}\newcommand{\logit}{\operatorname{logit}}\newcommand{\expit}{\mathtt{expit}}\newcommand{\cK}{\mathcal{K}}\newcommand{\dl}{\dot{l}}\newcommand{\dotp}{\dot{p}}\newcommand{\wl}{\wt{l}}\newcommand{\Gauss}{\mathrm{Gauss}}\newcommand{\fA}{\mathfrak{A}}\newcommand{\under}{\mathrm{under}\;}\newcommand{\whtheta}{\wh{\theta}}\newcommand{\Em}{\mathrm{Em}}\newcommand{\ztheta}{{\theta_0}}
\newcommand{\rO}{\mathrm{O}}\newcommand{\Bin}{\mathrm{Bin}}\newcommand{\rW}{\mathrm{W}}\newcommand{\rG}{\mathrm{G}}\newcommand{\rB}{\mathrm{B}}\newcommand{\rU}{\mathrm{U}}\newcommand{\HG}{\mathrm{HG}}\newcommand{\GAMMA}{\mathrm{Gamma}}\newcommand{\Cauchy}{\mathrm{Cauchy}}\newcommand{\rt}{\mathrm{t}}\newcommand{\rF}{\mathrm{F}}
\newcommand{\FE}{\mathrm{FE}}\newcommand{\bV}{\boldsymbol{V}}\newcommand{\GLS}{\mathrm{GLS}}\newcommand{\be}{\boldsymbol{e}}\newcommand{\POOL}{\mathrm{POOL}}\newcommand{\GMM}{\mathrm{GMM}}\newcommand{\MM}{\mathrm{MM}}\newcommand{\SSIV}{\mathrm{SSIV}}\newcommand{\JIV}{\mathrm{JIV}}\newcommand{\AR}{\mathrm{AR}}\newcommand{\ILS}{\mathrm{ILS}}\newcommand{\SLS}{\mathrm{SLS}}\newcommand{\LIML}{\mathrm{LIML}}

\newcommand{\Rad}{\mathrm{Rad}}\newcommand{\bY}{\boldsymbol{Y}}\newcommand{\pone}{{(1)}}\newcommand{\ptwo}{{(2)}}\newcommand{\ps}[1]{{(#1)}}\newcommand{\fsub}{\overset{\text{finite}}{\subset}}


\newcommand{\varlim}{\varprojlim}\newcommand{\Hom}{\mathrm{Hom}}\newcommand{\Iso}{\mathrm{Iso}}\newcommand{\Mor}{\mathrm{Mor}}\newcommand{\Isom}{\mathrm{Isom}}\newcommand{\Aut}{\mathrm{Aut}}\newcommand{\End}{\mathrm{End}}\newcommand{\op}{\mathrm{op}}\newcommand{\ev}{\mathrm{ev}}\newcommand{\Ob}{\mathrm{Ob}}\newcommand{\Ar}{\mathrm{Ar}}\newcommand{\Arr}{\mathrm{Arr}}\newcommand{\Set}{\mathrm{Set}}\newcommand{\Grp}{\mathrm{Grp}}\newcommand{\Cat}{\mathrm{Cat}}\newcommand{\Mon}{\mathrm{Mon}}\newcommand{\Ring}{\mathrm{Ring}}\newcommand{\CRing}{\mathrm{CRing}}\newcommand{\Ab}{\mathrm{Ab}}\newcommand{\Pos}{\mathrm{Pos}}\newcommand{\Vect}{\mathrm{Vect}}\newcommand{\FinVect}{\mathrm{FinVect}}\newcommand{\FinSet}{\mathrm{FinSet}}\newcommand{\FinMeas}{\mathrm{FinMeas}}\newcommand{\OmegaAlg}{\Omega\text{-}\mathrm{Alg}}\newcommand{\OmegaEAlg}{(\Omega,E)\text{-}\mathrm{Alg}}\newcommand{\Fun}{\mathrm{Fun}}\newcommand{\Func}{\mathrm{Func}}

\newcommand{\Stoch}{\mathrm{Stoch}}\newcommand{\FinStoch}{\mathrm{FinStoch}}\newcommand{\Copy}{\mathrm{copy}}\newcommand{\Delete}{\mathrm{delete}}
\newcommand{\Bool}{\mathrm{Bool}}\newcommand{\CABool}{\mathrm{CABool}}\newcommand{\CompBoolAlg}{\mathrm{CompBoolAlg}}\newcommand{\BoolAlg}{\mathrm{BoolAlg}}\newcommand{\BoolRng}{\mathrm{BoolRng}}\newcommand{\HeytAlg}{\mathrm{HeytAlg}}\newcommand{\CompHeytAlg}{\mathrm{CompHeytAlg}}\newcommand{\Lat}{\mathrm{Lat}}\newcommand{\CompLat}{\mathrm{CompLat}}\newcommand{\SemiLat}{\mathrm{SemiLat}}\newcommand{\Stone}{\mathrm{Stone}}\newcommand{\Mfd}{\mathrm{Mfd}}\newcommand{\LieAlg}{\mathrm{LieAlg}}
\newcommand{\Op}{\mathrm{Op}}
\newcommand{\Sh}{\mathrm{Sh}}
\newcommand{\Diff}{\mathrm{Diff}}
\newcommand{\B}{\mathcal{B}}\newcommand{\cB}{\mathcal{B}}\newcommand{\Span}{\mathrm{Span}}\newcommand{\Corr}{\mathrm{Corr}}\newcommand{\Decat}{\mathrm{Decat}}\newcommand{\Rep}{\mathrm{Rep}}\newcommand{\Grpd}{\mathrm{Grpd}}\newcommand{\sSet}{\mathrm{sSet}}\newcommand{\Mod}{\mathrm{Mod}}\newcommand{\SmoothMnf}{\mathrm{SmoothMnf}}\newcommand{\coker}{\mathrm{coker}}\newcommand{\Ord}{\mathrm{Ord}}\newcommand{\eq}{\mathrm{eq}}\newcommand{\coeq}{\mathrm{coeq}}\newcommand{\act}{\mathrm{act}}

\newcommand{\apf}{\mathrm{apf}}\newcommand{\opt}{\mathrm{opt}}\newcommand{\IS}{\mathrm{IS}}\newcommand{\IR}{\mathrm{IR}}\newcommand{\iidsim}{\overset{\text{i.i.d.}}{\sim}}\newcommand{\propt}{\,\propto\,}\newcommand{\bM}{\mathbb{M}}\newcommand{\cX}{\mathcal{X}}\newcommand{\cY}{\mathcal{Y}}\newcommand{\cP}{\mathcal{P}}\newcommand{\ola}[1]{\overleftarrow{#1}}

\renewcommand{\iff}{\;\mathrm{iff}\;}
\newcommand{\False}{\mathrm{False}}\newcommand{\True}{\mathrm{True}}
\newcommand{\otherwise}{\mathrm{otherwise}}
\newcommand{\suchthat}{\;\mathrm{s.t.}\;}

\newcommand{\cM}{\mathcal{M}}\newcommand{\M}{\mathbb{M}}\newcommand{\cF}{\mathcal{F}}\newcommand{\cD}{\mathcal{D}}\newcommand{\fX}{\mathfrak{X}}\newcommand{\fY}{\mathfrak{Y}}\newcommand{\fZ}{\mathfrak{Z}}\renewcommand{\H}{\mathcal{H}}\newcommand{\cH}{\mathcal{H}}\newcommand{\fH}{\mathfrak{H}}\newcommand{\bH}{\mathbb{H}}\newcommand{\id}{\mathrm{id}}\newcommand{\A}{\mathcal{A}}
\newcommand{\lmd}{\lambda}
\newcommand{\Lmd}{\Lambda}
\newcommand{\cI}{\mathcal{I}}

\newcommand{\Lrarrow}{\;\;\Leftrightarrow\;\;}
\DeclareMathOperator{\des}{des}
\DeclareMathOperator{\nd}{nd}
\DeclareMathOperator{\dsep}{d-sep}
\DeclareMathOperator{\sep}{sep}
\newcommand{\rLL}{\mathrm{LL}}\newcommand{\HT}{\mathrm{HT}}\newcommand{\PS}{\mathrm{PS}}\newcommand{\rI}{\mathrm{I}}
$$

:::

:::



* ただし，簡単のため，目標分布は標準 Gauss 分布 $N_d(0,I_d)$ とする：
  $$
  \pi^d(x)\,\propto\,e^{-U^d(x)},\qquad U^d(x)=\frac{\norm{x}^2}{2}.
  $$
* 「高次元に強い」とは，次のように決めるものとする：
  * 計算複雑性（例えば１つの独立サンプルを出力するのに要する $U(x),\nabla U(x)$ 評価の回数）のスケール $O(d^{\alpha})$ の $\alpha$ が小さいと「強い」とする．
  * 同じスケールにおいては，同一回数の $U(x),\nabla U(x)$ 評価（近似的に CPU 時間と比例するものとする）^[シミュレーション法が定まっていないのでうまく言えないが，理想的なアルゴリズムではほとんどその通りになる．]で，より多くの独立サンプルを生み出す方が「強い」とする．

正解は本記事の最後に触れる．

それまでは，上の４つの gif が表す確率過程が，どのように生成されているか解説する．

`PDMPFlux.jl` の実装について詳しくなってもらうと同時に，６つの PDMP 手法を通じて，「より速い手法を作るにはどういうアイデアがあり得るか？」を一緒に考えて欲しい．実際，マジのアイデア不足である．

最後に，現状での理論が与える示唆を少し紹介する．

## PDMP の基本：Poisson 点過程を打つ

基本的に，全ての PDMP アルゴリズムは点を打っている（＝特定の Poisson 点過程をシミュレーションしている）だけである．この時点で，既存の MCMC アルゴリズムとは全く違う．^[だが，最終的に最もうまくいく MCMC アルゴリズムと，最もうまくいく PDMP アルゴリズムは，確率過程としてはとても似通ってくる．シミュレーション方法が違うのみである．]

![PDMP アルゴリズム＝緑の点がどこに来るかをシミュレーションする．緑の点で「どのように方向転換をするか」は，手法ごとに違う．](Emergence/ZZS_demo.gif){fig-align="center" width="50%"}

打った点の間を直線で（または他の規定の規則で）補間するだけで最終的な output が完成する（次のコードの `sample_from_skeleton` の部分）．

```{.julia filename="PDMPFlux.jl/src/sample.jl"}
function sample(
    sampler::AbstractPDMP,  # PDMP サンプラーの種類を指定
    N_sk::Int,  # シミュレーションするイベントの数
    N_samples::Int,  # 最終的に出力するサンプルの数
    xinit::Vector{Float64},  # 初期値（位置）
    vinit::Vector{Float64};  # 初期値（速度）
    seed::Int=nothing,  # 乱数のシード（必要ならば）
    verbose::Bool = true  # 進捗バーを表示するかどうか
)::Matrix{Float64}

    history = sample_skeleton(sampler, N_sk, xinit, vinit, seed=seed, verbose=verbose)
    return sample_from_skeleton(sampler, N_samples, history)

end
```

この `sample_skeleton` が PDMP シミュレーションの本質であり，中では「次のイベント時刻の特定」の繰り返しである．

仮に１つ前のイベント時刻 $T_{i-1}$ で $(X_{T_{i-1}},V_{T_{i-1}})=(x,v)\in\R^{2d}$ まで来ているとしよう．すると，次のイベント時刻 $T_i$ が来るまでの時間 $T_i-T_{i-1}$ は，強度関数
$$
\lambda(t)=\max(0,\brac{V_t|\nabla U(X_t)})
$$ {#eq-lambda}
で定まる．

強度関数 (intensity function) とは Poisson 点過程の双対可予測射影を指す語であるが，生存解析におけるハザード関数と捉えても良い．

結局，次のイベント時刻 $T_i$ は
$$
\P[T_i-T_{i-1}\ge t|T_{i-1}]=\exp\paren{-\int^t_0\lambda(s)\,ds}
$$
に従って定まる．

$\lambda$ が定数ならば，到着時間 $T_i-T_{i-1}$ は指数分布である．しかし，式 ([-@eq-lambda]) は「速度 $V_t$ の，確率密度が下がっていく方向 $\nabla U(X_t)$ の成分が大きければ大きいほどイベントが起きやすい」ことを意味する．

これにより，式 ([-@eq-lambda]) に従ってイベント時刻を決定し，そこでうまく高密度領域に戻るような「方向転換」をデザインすれば，サンプラーを $U(x)$ の値が大きい領域に誘導することができる．

これが，確率分布 $\pi(x)\propto e^{-U(x)}$ に収束する PDMP の作り方である．

"PDMP" という名前についてであるが，直線部分が "piecewise deterministic" と呼ばれるゆえんであり，"Markov process" の部分は主に，方向転換が Markov 点過程に従うことと，方向転換の仕方がランダムである（場合がある）ことを指す．

## PDMP サンプラー戦国時代：方向転換のデザイン

最も代表的な手法は BPS (Bouncy Particle Sampler) 群である．

これは Snell の反射の法則の通りに方向転換をする．つまり，イベントが起こった場合，ポテンシャル $U(x)=-\log\pi(x)$ の等高線の接線を鏡に見立てて，反射する．式で表すと：
$$
V_t^{\text{new}}\gets V_t-2\frac{\brac{V_t|\nabla U(X_t)}}{\norm{\nabla U(X_t)}^2}\nabla U(X_t).
$$

これにより速度 $V_t$ のうち $\nabla U(X_t)$ に平行だった成分は反射され，垂直な成分はそのまま残る．

:::: {layout-ncol="2"}

![[@Bouchard-Cote+2018]](../../2024/Slides/PDMPs/BPS_SlantedGauss2D.gif){width=50%}

![[@Bierkens+2020]](../../2024/Slides/PDMPs/Boomerang_SlantedGauss2D.gif){width=50%}

::::

右側の Boomerang Sampler では参照測度が違う．BPS と同じメカニズムだが，$\R^n$ 上の Lebesgue 測度に対して設計する代わりに，標準 Gauss 測度に対して設計されている．このため，$\R^{\infty}$ 上でも定義可能な手法になっている．^[$\R^{\infty}$ または一般の可分 Hilbert 空間上では Lebesgue 測度は存在しないが，Gauss 測度は存在し続ける．] このような手法は function space MCMC とも呼ばれる [@Cotter+2013]．

この BPS 群の最大の欠点は refreshment という追加の動作が必要な点である．定期的に速度 $V_t$ を平衡分布にリセットする必要がある．

さもなくば，ダイナミクスのランダムネスが不足し，エルゴード性が失われたり，すごく遅くなったりする：


In [ ]:
#| eval: false
#| code-fold: true

using PDMPFlux, Plots

@inline function ∇U(x::AbstractVector)
  return x
end

dim = 2
sampler = BPS(dim, ∇U; refresh_rate=0.0)

N_sk, xinit, vinit = 1_000, randn(dim), randn(dim)
vinit = vinit ./ sqrt(sum(vinit.^2))

output_BPS = sample_skeleton(sampler, N_sk, xinit, vinit, seed=20250428)
x_BPS = output_BPS.X
v_BPS = output_BPS.V
t_BPS = output_BPS.t

anim_traj(output_BPS, 100; plot_start=50, filename="BPS.gif", color="#E95420", scatter_color="#78C2AD", background="#F0F1EB", title="BPS without Refreshment", xlabel="", ylabel="", axis=false)

![](Emergence/BPS.gif){fig-align="center" width="50%"}

この問題を迂回する１つの方法は，そもそも進行方向を $\{\pm1\}^d$ に制限することである．

![[@Bierkens+2019]](../../2024/Slides/PDMPs/ZigZag_SlantedGauss2D.gif){width=30%}

これは trajectory の形状から Zig-Zag sampler と呼ばれるが，異方性に弱いという欠点がある．

つまり，分布の形状が $\{\pm1\}^d$ にうまく沿っていない場合，大変非効率的な移動を強いられるという欠点がある．


In [ ]:
#| eval: false
#| code-fold: true

using PDMPFlux, LinearAlgebra

const γ = 10

@inline function ∇U(x::AbstractVector)
  return [γ,1/γ] ⋅ x
end

dim = 2
sampler = ZigZag(dim, ∇U; refresh_rate=0.0)

N_sk, xinit, vinit = 1_000, zeros(dim), ones(dim)

output_ZZ = sample_skeleton(sampler, N_sk, xinit, vinit, seed=20250428)
x_ZZ = output_ZZ.X
v_ZZ = output_ZZ.V
t_ZZ = output_ZZ.t

anim_traj(output_ZZ, 100; plot_start=50, filename="ZZS.gif", color="#E95420", scatter_color="#78C2AD", background="#F0F1EB", title="Zig-Zag on an Anisotropic Target", xlims=(-5, 5), ylims=(-5, 5))

![](Emergence/ZZS.gif){fig-align="center" width="50%"}

## PDMP の最近の発展

以上，今まで見てきたどの手法も得手・不得手というものがあった．

### FECMC：確率的反射，部分的リフレッシュ

今までの手法は全て，方向転換の仕方がシンプルで，ランダムネスは（実は）ない．

Forward Event-Chain Monte Carlo 法 [@Michel+2020] は，「ランダムな反射」を設計することで，「リフレッシュ動作が欠かせない」という先述の BPS の欠点を克服した．

特に「絶対に今来た道を戻らない "Forward"」という設計が凝らされており，次のようにくるくる回る軌道を描く：

![[@Michel+2020]](../../2024/Slides/PDMPs/ForwardECMC_StandardGauss2D.gif){fig-align="center" width=30%}

名前についてであるが，ECMC (Event-Chain Monte Carlo) 法とは，実は PDMP という名前がついて統計に輸入される前の，物理学で用いられていた手法である [@Bernard+2009], [@Michel+2014], [@Nishikawa+2015], [@Krauth2021]．

そもそも PDMP とは ECMC などの名前で，はじめて氷の融解の過程の全原子シミュレーションに成功（！）して [@Bernard-Krauth2011]，満を持して統計分野に輸入されたのであった．^[他にも直接の祖先は ECMC 以外にも辿れる [@Peters-deWith2012]．]

### Sticky PDMP：アトムを含む分布への解決策

PDMP はなんといっても連続時間の軌道を出力するという点に，過去の Monte Carlo 法にない特徴がある．

この特徴が極めて不思議な応用をもたらしてくれることを，コミュニティは近年自覚しつつある．

まず，目標の確率分布は絶対連続じゃなく，アトムが有限個含まれていても良い．

例えば，変数選択のことを考えて，事後分布が $\alpha\pi(x)+(1-\alpha)\delta_0(dx)$ という形で表せたとしよう．

この $\alpha$ の値（回帰で言えば，当該変数の事後包含確率）の値と，モデルに含まれた場合の事後分布を同時にサンプリングしたい場合，Sticky Zig-Zag sampler [@Bierkens+2023] が使える：

![[@Bierkens+2023]](../../2024/Slides/PDMPs/StickyZigZag.gif){fig-align="center" width=30%}

### Speed-Up PDMP：加速による重点サンプリング

連続時間のアルゴリズムになったため，「加速」という概念が生まれる．

単に加速するのではなく，位置に応じてサンプラーの速さを変えることで，分布の「どこを重視して欲しいか」を指定する使い方ができる．

これにより，裾が重い分布における尾部領域や，多峰性分布における谷の部分など，従来の MCMC では探索が困難であった領域に，違った重みを与えることができる．

![[@Vasdekis-Roberts2023]](../../2024/Slides/PDMPs/SUZZ_SlantedGauss2D.gif){fig-align="center" width=30%}

[@Bertazzi-Vasdekis2025] では，なんと 13 成分の混合 Gauss 分布から，ひとつもモードを潰さない正確なサンプリングを達成していることを報告している：

![[@Bertazzi-Vasdekis2025 p.19 Figure 4]](Emergence/Bertazzi_Vasdekis.png){fig-align="center" width=50%}

## クイズの答え

正解は Forward Event-Chain Monte Carlo である．

![[@Michel+2020]](../../2024/Slides/PDMPs/ForwardECMC_StandardGauss2D.gif){fig-align="center" width=30%}

ただし，ほとんどのサンプラーには，ターゲット分布が factorized な形 $\pi(x)=\prod_{f\in F}\pi_f(x_f)$ で書ける場合にだけ使える効率的な実装法があり，これを採用した場合のスケーリングは HMC も凌ぐ [@Krauth2024]．

実はこの局所性を活かした実装が，ECMC という名前で，氷などの凝縮系の Monte Carlo シミュレーションで圧倒的な性能を見せつけてきた理由である．この点は [@Tartero-Krauth2023] などが良い入門になる．

ここでは BPS, Zig-Zag, FECMC の，一般のターゲット分布に対して通用する実装で比較してみる（`PDMPFlux.jl` のデフォルト実装）．

例えば，テスト関数 $h(x)=\|x\|^2$ の期待値を推定する際の二乗誤差を比較してみると，FECMC の方が効率が良さがわかる：


In [ ]:
#| eval: false
#| code-fold: true

using PDMPFlux, Plots, ProgressBars, LinearAlgebra, StatsPlots

const dim = 100

@inline function ∇U(x::AbstractVector)
  return x
end

@inline function ∇U_Boomerang(x::AbstractVector)
  return ones(dim)
end

function h_estimate_online(
  sampler::PDMPFlux.AbstractPDMP,
  T_end::Float64,
  xinit::Vector{Float64},
  vinit::Vector{Float64};
  seed::Union{Int, Nothing}=nothing,
)::Float64
  if !(isfinite(T_end)) || T_end < 0
    throw(ArgumentError("T_end must be finite and non-negative. Current value: $T_end"))
  end
  d = length(xinit)
  if d == 0
    throw(ArgumentError("xinit must be non-empty"))
  end
  if T_end == 0.0
    return 0.0
  end

  state = PDMPFlux.init_state(sampler, xinit, vinit, seed)

  estimate = 0.0

  # 現在のイベント点（post-jump）を保持し、次イベントまでの区間寄与を足す
  t_prev = state.t
  x_prev = copy(state.x)
  v_prev = copy(state.v)

  while t_prev < T_end
    PDMPFlux.get_event_state!(state, sampler)  # mutates state in-place; advances state.t to next accepted event
    t_next = state.t

    if t_next <= T_end
      Δt = t_next - t_prev
      # 区間 [t_prev, t_next] は (x_prev, v_prev) からの flow
      nx = dot(x_prev, x_prev)
      xv = dot(x_prev, v_prev)
      estimate += nx * Δt + (Δt^3) / 3 + (Δt^2) * xv

      t_prev = t_next
      copyto!(x_prev, state.x)
      copyto!(v_prev, state.v)
    else
      # overshoot: 最後は flow で t=T_end の点を作ってそこで打ち切る
      Δt = T_end - t_prev
      nx = dot(x_prev, x_prev)
      xv = dot(x_prev, v_prev)
      estimate += nx * Δt + (Δt^3) / 3 + (Δt^2) * xv
      break
    end
  end

  return (estimate - d * T_end) / sqrt(d) / T_end  # CAUTION: inprecise denominator (intentionally)
end

function Experiment_once()
  ZZS = ZigZag(dim, ∇U; refresh_rate=0.0)
  T, xinit, vinit = Float64(1000), randn(dim), ones(dim)
  t_ZZ = @elapsed output_ZZ = h_estimate_online(ZZS, T, xinit, vinit)

  BPS_sampler = BPS(dim, ∇U; refresh_rate=1.4)
  vinit = randn(dim)
  vinit = vinit ./ norm(vinit)
  t_BPS = @elapsed output_BPS = h_estimate_online(BPS_sampler, T, xinit, vinit)

  vinit = randn(dim)
  vinit = vinit ./ norm(vinit)
  FECMC = ForwardECMC(dim, ∇U)
  t_ForwardECMC = @elapsed output_ForwardECMC = h_estimate_online(FECMC, T, xinit, vinit)
  return [abs2(output_ZZ) * t_ZZ, abs2(output_BPS) * t_BPS, abs2(output_ForwardECMC) * t_ForwardECMC]
end

function Experiment(iter::Int=100)
  results = Matrix{Float64}(undef, iter, 3)
  for i in ProgressBar(1:iter)
    results[i,:] = Experiment_once()
  end
  return results
end

results = Experiment(100)

iter  = size(results, 1)
labels = ["ZZS", "BPS", "FECMC"]
groups = repeat(labels, inner=iter)
groups = categorical(groups; ordered=true, levels=labels)
boxplot(groups, vec(results);
    group = groups,
    bar_width = 0.7,
    legend = :bottomleft,
    xlabel = "Sampler",
    ylabel = "Squared Error × Computational Time",
    yscale = :log10)

![](Emergence/Comparison.png){fig-align="center" width="50%"}

縦軸は実行時間によって調節した平均自乗誤差 (MSE) であり，小さいほど性能が良い．

この性能差はスケーリング極限／拡散近似の数学によって理論的に説明することができる．

## 参考文献